<div>
    <h1 align="center">Tabular Playground Series - Jun 2021</h1>
    <h2 align="center">Ensembling & Comparative Method for Classifier</h2>
    <h4 align="center">By: Somayyeh Gholami & Mehran Kazeminia</h4>
</div>

<div class="alert alert-success">  
</div>

## Description:

#### In Kaggle challenges as well as real projects, we usually calculate the average (Weighted average) of the results of several different methods  to get a better score. This method is called "Ensembling" and is very common.

#### **What does it mean if the new results get a better score? Did all the hundreds of thousands of rows get better results? Certainly not.**

#### The fact is that only the results of some rows are better and the results of others are worse. However, we see that the overall result has improved.

#### Here we want to identify rows whose new results are probably not better. Of course, after identifying these rows, we return the results to their pre-"Ensembling" state and we will see that by doing so, our score will be even better.

#### We call this method "Comparative Method" and it is our initiative. The "Comparative Method" can be used for both regression and classifier. But of course there are differences in the way it is done on different issues.

#### For more information, refer to the following address:

#### https://www.kaggle.com/c/tabular-playground-series-jun-2021/discussion/246031


<div class="alert alert-success">  
</div>

## Import & Data Set

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df1 = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
df2 = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
sam = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

In [ ]:
sub1 = pd.read_csv('../input/1-tps-jun-21-histgradient-catboost-nn/submission1.csv')
# Public Score: 1.75770
# HistGradientBoostingClassifier
# display(sub1)

In [ ]:
sub2 = pd.read_csv('../input/1-tps-jun-21-histgradient-catboost-nn/submission2.csv')
# Public Score: 1.75011
# CatBoostClassifier
# display(sub2)

In [ ]:
sub3 = pd.read_csv('../input/1-tps-jun-21-histgradient-catboost-nn/submission3.csv')
# Public Score: 1.74587
# Neural Networks
# display(sub3)

Thanks to: @oxzplvifi https://www.kaggle.com/oxzplvifi/tabular-residual-network

In [ ]:
sub4 = pd.read_csv('../input/1-tps-jun-21-histgradient-catboost-nn/submission4.csv')
# Public Score: 1.74522
# tabular residual network
# display(sub4)

Thanks to: @bhavikjain https://www.kaggle.com/bhavikjain/tps-june-21-eda-models

In [ ]:
sub5 = pd.read_csv('../input/1-tps-jun-21-histgradient-catboost-nn/submission5.csv')
# Public Score: 1.74456
# EDA + Models
# display(sub5)

Thanks to: @fusioncenter https://www.kaggle.com/fusioncenter/residual-network-for-tabular-data

In [ ]:
sub6 = pd.read_csv('../input/1-tps-jun-21-histgradient-catboost-nn/submission6.csv')
# Public Score: 1.74442
# Residual network
# display(sub6)

<div class="alert alert-success">  
</div>

## Ensembling

In [ ]:
def generate(main, support, coeff):
    
    g = main.copy()    
    for i in main.columns[1:]:
        
        res = []
        lm, Is = [], []        
        lm = main[i].tolist()
        ls = support[i].tolist()  
        
        for j in range(len(main)):
            res.append((lm[j] * coeff) + (ls[j] * (1.- coeff)))            
        g[i] = res
        
    return g

In [ ]:
sub = generate(sub2, sub1, 0.85)

sub = generate(sub3, sub , 0.85)

sub = generate(sub4, sub , 0.85)

sub = generate(sub5, sub , 0.85)

sub = generate(sub6, sub , 0.55)

sub_ens = sub

In [ ]:
display(sub_ens, sub_ens.describe().transpose())

In [ ]:
sub_ens.to_csv("submission_ens.csv",index=False)
# Public Score: 1.74415
!ls

<div class="alert alert-success">  
</div>

## Comparative Method for Classifier

In [ ]:
def improve(sub1, sub2, sub3, sub4, sub5, sub6, sub_ens, majority, m_majority):  
    
    sub1v = sub1.values
    sub2v = sub2.values
    sub3v = sub3.values
    sub4v = sub4.values
    sub5v = sub5.values 
    sub6v = sub6.values
    
    imp = sub_ens.copy()
    impv = imp.values
    NCLASS = 9
    number = 0

    for i in range (len(sub_ens)):
        c_count = 0  
        row = impv[i,1:]
        row_sort = np.sort(row)        
        
        row1 = sub1v[i,1:]
        row2 = sub2v[i,1:]
        row3 = sub3v[i,1:]
        row4 = sub4v[i,1:]
        row5 = sub5v[i,1:]    
        row6 = sub6v[i,1:] 
        row1_sort = np.sort(row1)
        row2_sort = np.sort(row2)
        row3_sort = np.sort(row3)
        row4_sort = np.sort(row4)
        row5_sort = np.sort(row5)
        row6_sort = np.sort(row6)
                      
        for j in range (NCLASS): 
            count = 0
            
            for k in range (NCLASS):                
                if (row6[j] == row6_sort[k]): 
                    
                    if (row1[j] == row1_sort[k]):
                        count = count + 1
                    if (row2[j] == row2_sort[k]):
                        count = count + 1                   
                    if (row3[j] == row3_sort[k]):
                        count = count + 1   
                    if (row4[j] == row4_sort[k]):
                        count = count + 1 
                    if (row5[j] == row5_sort[k]):
                        count = count + 1 
                        
            if (count >= majority):
                c_count = c_count + 1
        
        if ((c_count >= m_majority) and (row6_sort[8] >= row_sort[8])): 
            impv[i, 1:] = row6            
            number = number + 1            
#           print (number, i, c_count)                         
                    
    imp.iloc[:, 1:] = impv[:, 1:]
    p_number = round(((number / 100000) * 100),2)
    print('>>>  R  E  T  U  R  N  S  <<<')
    print(30 * '=')
    print(f'Number of changes: {number}\n')
    print(f'Percentage of changes: {p_number} %')
    print(30 * '=')
    return imp      

In [ ]:
sub_imp = improve(sub1, sub2, sub3, sub4, sub5, sub6, sub_ens, 5, 7)

In [ ]:
display(sub_imp, sub_imp.describe().transpose())

In [ ]:
sub_imp.to_csv("submission_imp.csv",index=False)
# Public Score: 1.74408
!ls

<div class="alert alert-success">
    <h1 align="center">If you find this work useful, please don't forget upvoting :)</h1>
</div>